# Fiber photometry data conversion to NWB

This notebook demonstrates how to convert an example session from the Fiber photometry dataset to NWB.

## Fiber photometry dataset

This dataset contains fiber photometry recordings during decision-making behavioral task in rats. Deeplabcut software (v.2.2.3) was used for tracking the behavior ports (right port, central port, and left port) and 6 body parts (right ear, nose, left ear, mid-point along the right torso, mid-point along the left torso, and base of the tail). Video data were acquired using cameras attached to the ceiling of behavior rigs to capture the top-down view of the arena (Doric USB3 behavior camera, Sony IMX290, recorded with Doric Neuroscience Studio v6 software). The fluorescence from activity-dependent (GRAB-DA and GRAB-ACh) and activity-independent (isosbestic or mCherry) signals was acquired simultaneously via demodulation and downsampled on-the-fly by a factor of 25 to ~481.9 Hz. The recorded demodulated fluorescence was corrected for photobleaching and motion using Two-channel motion artifact correction (3) with mCherry or isosbestic signal as the activity-independent channel. The behavioral tasks were conducted in a high-throughput facility where rats were trained in increasingly complex protocols. Trials were initiated by a nose-poke in a lit center port and required maintaining a center fixation for 0.8 to 1.2 seconds, during which a tone indicated the possible reward size. A subsequent side LED indicated the potential reward location, followed by a delay period drawn from an exponential distribution (mean = 2.5 s). Rats could opt out at any time by poking the unlit port, restarting the trial. Catch trials, where the delay period only ended if the rat opted out, constituted 15-25% of the trials. Rats received penalties for premature fixation breaks. Additionally, the tasks introduced semi-observable hidden states by varying reward statistics across uncued blocks (high, low, and mixed), structured hierarchically, with blocks transitioning after 40 successfully completed trials.


This notebook demonstrates how to convert an example session to NWB.

This dataset have the following data streams:
- Raw behavior: Bpod output (.mat), Video (.mp4)
- Processed behavior: DeepLabCut output (.h5)
- Fiber photometry: Doric (.doric or .csv)

## Notes on the conversion

The conversion notes is located in `src/constantinople_lab_to_nwb/fiber_photometry/fiber_photometry_notes.md`. This file contains information about the expected file structure and the conversion process.


## Convert a single session to NWB

The `fiber_photometry_convert_session.py` script defines the `session_to_nwb` function that converts a session of fiber photometry data to NWB.

In [ ]:
from pathlib import Path
from constantinople_lab_to_nwb.fiber_photometry.fiber_photometry_convert_session import session_to_nwb
from neuroconv.utils import load_dict_from_file, dict_deep_update

from constantinople_lab_to_nwb.fiber_photometry import FiberPhotometryNWBConverter
from ndx_pose import PoseEstimation

from constantinople_lab_to_nwb.utils import get_subject_metadata_from_rat_info_folder

# Parameters for conversion

# The path to the raw fiber photometry file (.doric or .csv).
doric_file_path="/Volumes/T9/Constantinople/Preprocessed_data/J069/Raw/J069_ACh_20230809_HJJ_0002.doric"
# The path to the NWB file to write.
nwbfile_path="/Volumes/T9/Constantinople/nwbfiles/J069_ACh_20230809_HJJ_0002.nwb"
# The path to the DLC output (.h5), optional
dlc_file_path="/Volumes/T9/Constantinople/DeepLabCut/J069/J069-2023-08-09_rig104cam01_0002compDLC_resnet50_GRAB_DA_DMS_RIG104DoricCamera_J029May12shuffle1_500000.h5"
# The path to the behavior camera recording (.mp4), optional
video_file_path="/Volumes/T9/Constantinople/Compressed Videos/J069/J069-2023-08-09_rig104cam01_0002comp.mp4"
# The raw behavior data from Bpod (contains data for a single session)
bpod_file_path="/Volumes/T9/Constantinople/raw_Bpod/J069/DataFiles/J069_RWTautowait2_20230809_131216.mat"

# Load the default fiber photometry metadata from the yaml file
# For .doric files use "doric_fiber_photometry_metadata.yaml"
# For .csv files use "doric_csv_fiber_photometry_metadata.yaml"
current_dir = Path(__file__).parent if '__file__' in globals() else Path.cwd().parent
fiber_photometry_metadata_file_path = current_dir / "metadata" / "doric_fiber_photometry_metadata.yaml"
fiber_photometry_metadata = load_dict_from_file(fiber_photometry_metadata_file_path)

# Get subject metadata from rat registry
rat_registry_folder_path = "/Volumes/T9/Constantinople/Rat_info"
subject_metadata = get_subject_metadata_from_rat_info_folder(
    folder_path=rat_registry_folder_path,
    subject_id="J069",
    date="2023-08-09",
)

# Optional parameters

# Whether to overwrite an existing NWB file.
overwrite = True
# Whether to print verbose output.
verbose = True

session_to_nwb(
    nwbfile_path=nwbfile_path,
    raw_fiber_photometry_file_path=doric_file_path,
    fiber_photometry_metadata=fiber_photometry_metadata,
    raw_behavior_file_path=bpod_file_path,
    subject_metadata=subject_metadata,
    dlc_file_path=dlc_file_path,
    video_file_path=video_file_path,
    overwrite=overwrite,
    verbose=verbose,
)